# Databases

## Problem Space
A database is locked in a struggle against the hard drive

__Obligatory write-speed comparison of various I/O devices:__
SATA bus == 750MB/second
PCI 3.0 bus == 40GB/second (53.3x faster)
Memory bus == 14.9GB/second/channel (75x faster)

A 7200RPM disk can only perform 1 seek/4ms, or 250 seeks/second. An application relying on seeking once per operation is thus bounded at 250 ops/second. This ignores long read times and other application work.

SSDs can seek 60x faster than a hard disk
$seek/4ms * 60seeks = 60 seeks/4ms = 15 seeks/ms$

However, their storage cells have a fixed number of writes in their lifetime. 

Many optimizations go into spreading these write operations across all cells; garbage collection, bookkeeping(?), and presumably some kind of write scheduler.

Since writing to a disk is so slow, and memory is fast, maintaing a mirror image of recently-accessed data in memory; this is a __page cache__ (or __buffer cache__). Use a scheduler to find slow points in writes, and persist the in-memory data to disk.

__Watch out!__ Your data's durability in event of power, disk, or kernel failure is _very sensitive_ to the DB's page cache persistence algorithm!

## Evaluation Framework
1. Data Model
1. API
1. Transactions
1. Persistence
1. Indexing

### Data Model

Basically three kinds of data
1. Relational
1. Key-Value
1. Hierarchical

#### Relational Data
Create arbitrary, tables with columns representing types of data. Link these tables together. 

This set of tables and their relations is commonly referred to as a _schema_. Schema allows you to provide consistency checks. I don't know what that means, but I imagine something like this: This data should only persist if this data does; this data must look like this, or be within this range.)

Relational Database Management Systems (RDBMS) are really good at minimizing data duplication in a database through _normalization_

__normalization__ Removing data redundancy through the use of links (foreign keys in most cases). _Is not necessarily a good thing_.

Fundamental trade-off: Save disk space in exchange for greater CPU & disk load (disk I/O).


#### Key-Value Data
Associate a key (some # of bytes) with a value (another # of bytes).

Key-value data is often _homogeneous_ and _highly replicated_, so we can heavily compress it before storing to disk.

A sub-set of relational features can be provided, but does so by leveraging even more key-value lookups.


#### Hierarchical Data
A.k.a _document data_

Verbatim:
> The hierarchical model tends to store all relevant data in a single record, which as delineations for multiple keys and values, where the values could be additional associations of keys and values.

This means:
* All data for a record is stored together. This can mean redundant data is stored.
* Queries are simpler: All the data is stored together!

This makes it _highly_ flexible. It also most closely mirrors data structures in your application.


### API

Is it _in process_ or _out of process_?

__In process__ DB is running in the same process space as the client application. SQLite might fall in this category?

__Out-of-process__ Using a protocol to communicate with the database. Typical client/server interactions. Did you have to open a connection across TCP/IP? It's out of process.

Does it use SQL?